In [18]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request
from io import StringIO

## Extracting features of 2020 movies from Wikipedia

In [41]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [35]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [36]:
tables = soup.find_all('table',class_='wikitable sortable')

In [37]:
len(tables)

4

In [38]:
type(tables[0])

bs4.element.Tag

In [49]:
# Function to read HTML content using pd.read_html and StringIO
def read_html_from_soup_table(table):
    html_content = table.prettify()
    return pd.read_html(StringIO(html_content))[0]

# Apply the function to all tables
df1 = read_html_from_soup_table(tables[0])
df2 = read_html_from_soup_table(tables[1])
df3 = read_html_from_soup_table(tables[2])
df4 = read_html_from_soup_table(tables[3])

In [50]:
df = df1._append(df2._append(df3._append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [51]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House ...,Nicolas Pesce (director/screenplay); Andrea ...,[2]
1,J A N U A R Y,10,Underwater,20th Century Fox / TSG Entertainment / Che...,"William Eubank (director); Brian Duffield, A...",[3]
2,J A N U A R Y,10,Like a Boss,Paramount Pictures / Artists First,"Miguel Arteta (director); Sam Pitman, Adam Co...",[4]
3,J A N U A R Y,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazari...,NaN
4,J A N U A R Y,10,Inherit the Viper,Lionsgate / Barry Films / Tycor International...,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
...,...,...,...,...,...,...
270,D E C E M B E R,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R P...,Robert Rodriguez (director/screenplay); Priy...,[243]
271,D E C E M B E R,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke ...,[244]
272,D E C E M B E R,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screen...,[245]
273,D E C E M B E R,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey...,[246]


In [52]:
df_2020 = df[['Title','Cast and crew']]

In [53]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea ...
1,Underwater,"William Eubank (director); Brian Duffield, A..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Co..."
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazari...
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
...,...,...
270,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priy...
271,News of the World,Paul Greengrass (director/screenplay); Luke ...
272,One Night in Miami...,Regina King (director); Kemp Powers (screen...
273,Promising Young Woman,Emerald Fennell (director/screenplay); Carey...


In [54]:
!pip install tmdbv3api

In [59]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '8b7fe5e25350e17f99290c3ca6ecb62b'

In [60]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [61]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

TypeError: attribute name must be string, not 'int'

In [ ]:
df_2020

In [ ]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [ ]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

In [ ]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [ ]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [ ]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [ ]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [ ]:
df_2020

In [ ]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [ ]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
new_df20

In [ ]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [ ]:
new_df20.isna().sum()

In [ ]:
new_df20 = new_df20.dropna(how='any')

In [ ]:
new_df20.isna().sum()

In [ ]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

In [ ]:
new_df20

In [ ]:
old_df = pd.read_csv('final_data.csv')

In [ ]:
old_df

In [ ]:
final_df = old_df.append(new_df20,ignore_index=True)

In [ ]:
final_df

In [ ]:
final_df.to_csv('main_data.csv',index=False)